In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import pickle

dataset = pd.read_csv('spam.csv', encoding='latin-1')
dataset = dataset.rename(columns={"v1": "label", "v2": "message"})
dataset = dataset[['label', 'message']]
dataset['label'] = dataset['label'].map({'ham': 0, 'spam': 1})

X_train, X_test, y_train, y_test = train_test_split(
    dataset['message'], dataset['label'], test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

with open('spam_classifier.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

def predict_message(message):
    with open('spam_classifier.pkl', 'rb') as model_file:
        loaded_model = pickle.load(model_file)
    
    with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
        loaded_vectorizer = pickle.load(vectorizer_file)
    
    message_tfidf = loaded_vectorizer.transform([message])
    
    prediction = loaded_model.predict(message_tfidf)
    return "Spam" if prediction[0] == 1 else "Ham"

if __name__ == "__main__":
    user_input = input("Enter an SMS message to classify: ")
    result = predict_message(user_input)
    print(f"The message is classified as: {result}")


Accuracy: 0.9730941704035875
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       965
           1       1.00      0.80      0.89       150

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115

